In [5]:
from safetensors.torch import load_file

# Load tensors from the file
tensors = load_file("/Users/ammar3.shaikh/Desktop/ReLax/artifacts/weights/Llama-3.2-3B/model-00002-of-00002.safetensors")

# Inspect tensor keys
print(tensors.keys())

# Access a specific tensor

dict_keys(['model.layers.20.input_layernorm.weight', 'model.layers.20.mlp.down_proj.weight', 'model.layers.20.post_attention_layernorm.weight', 'model.layers.21.input_layernorm.weight', 'model.layers.21.mlp.down_proj.weight', 'model.layers.21.mlp.gate_proj.weight', 'model.layers.21.mlp.up_proj.weight', 'model.layers.21.post_attention_layernorm.weight', 'model.layers.21.self_attn.k_proj.weight', 'model.layers.21.self_attn.o_proj.weight', 'model.layers.21.self_attn.q_proj.weight', 'model.layers.21.self_attn.v_proj.weight', 'model.layers.22.input_layernorm.weight', 'model.layers.22.mlp.down_proj.weight', 'model.layers.22.mlp.gate_proj.weight', 'model.layers.22.mlp.up_proj.weight', 'model.layers.22.post_attention_layernorm.weight', 'model.layers.22.self_attn.k_proj.weight', 'model.layers.22.self_attn.o_proj.weight', 'model.layers.22.self_attn.q_proj.weight', 'model.layers.22.self_attn.v_proj.weight', 'model.layers.23.input_layernorm.weight', 'model.layers.23.mlp.down_proj.weight', 'model.l

In [1]:
import tiktoken
import base64

# 1. Pattern string from your Tokenizer
pat_str = r"(?i:'s|'t|'re|'ve|'m|'ll|'d)|[^\r\n\p{L}\p{N}]?\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]+[\r\n]*|\s*[\r\n]+|\s+(?!\S)|\s+"

# 2. Dummy model vocabulary content (base64 token -> rank)

    # A BPE ranks file where tokens align with pat_str behavior.
    # "token"   -> dG9rZW4=
    # "1"       -> MQ==
    # "2"       -> Mg==
    # " "       -> IA==
    # "b"       -> Yg==
    # "c"       -> Yw==
    # "a"       -> YQ== (Corrected from ZQ==)
    # "<"       -> PA==
    # "|"       -> fA==
    # "eot"     -> ZW90
    # "_"       -> Xw==
    # "id"      -> aWQ=
    # ">"       -> Pg==

dummy_model_content = """
dG9rZW4= 0
MQ== 1
Mg== 2
IA== 3
Yg== 4
Yw== 5
YQ== 6
PA== 7
fA== 8
ZW90 9
Xw== 10
aWQ= 11
Pg== 12
"""

# 3. Create mergeable_ranks by decoding base64 tokens
mergeable_ranks = {}
for line in dummy_model_content.strip().split("\n"):
    parts = line.split(" ")
    token_bytes_b64 = parts[0]
    rank = int(parts[1])
    mergeable_ranks[base64.b64decode(token_bytes_b64)] = rank

# 4. Create a tiktoken.Encoding instance (no special tokens needed for this test)
encoder = tiktoken.Encoding(
    name="test_dummy_vocab",
    pat_str=pat_str,
    mergeable_ranks=mergeable_ranks,
    special_tokens={} # Empty because we use allowed_special=set()
)

# 5. The string to test
test_string = "<|eot_id|>"

# 6. Encode with allowed_special=set() to treat it as normal text
encoded_ids = encoder.encode(test_string, allowed_special=set())

print(f"Input string: '{test_string}'")
print(f"Encoded IDs: {encoded_ids}")

# Optional: decode each token ID back to see the string parts
decoded_token_strings = [encoder.decode_single_token_bytes(token_id).decode('utf-8') for token_id in encoded_ids]
print(f"Decoded as token strings: {decoded_token_strings}")


Input string: '<|eot_id|>'
Encoded IDs: [7, 8, 9, 10, 11, 8, 12]
Decoded as token strings: ['<', '|', 'eot', '_', 'id', '|', '>']


In [4]:
import jax.numpy as jnp
import jax
key = jax.random.PRNGKey(0)
key1,key2,key3 = jax.random.split(key,3)

A = jax.random.normal(key1, (3,4))
B = jax.random.normal(key2, (4,5))
C = jax.random.normal(key3, (3,4,5))


In [6]:
D = jnp.einsum('ij,jk->ik',A,B)

check_D = jnp.matmul(A,B)

print(jnp.allclose(D,check_D))


True


In [8]:
E = jnp.einsum('ij,ij->ij',A,A)
check_E = A*A

print(jnp.allclose(E,check_E))

True


In [9]:
F = jnp.einsum('ij->',A)
check_F = jnp.sum(A)

print(jnp.allclose(F,check_F))


True


In [10]:
G = jnp.einsum('ij->j',A)
check_G = jnp.sum(A,axis=0)

print(jnp.allclose(G,check_G))



True


In [12]:
H = jnp.einsum('ij->i',A)
check_H = jnp.sum(A,axis=1)

print(jnp.allclose(H,check_H))

True


In [13]:
I = jnp.einsum('ij->ji',A)
check_I = A.T

print(jnp.allclose(I,check_I))

True
